In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time, json
import sys
sys.path.append("../")
sys.path.append("../chess_llm_interpretability")
import os
import torch
import numpy as np

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-10-22 19:32:49 __main__ INFO     torch.__version__='2.4.1+cu121', torch.version.cuda='12.1'


In [9]:
from src.models import ModelandTokenizer

# MODEL_KEY = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.1-8B-Instruct"

# MODEL_KEY = "meta-llama/Llama-3.2-3B"
# MODEL_KEY = "google/gemma-2-2b"
# MODEL_KEY = "meta-llama/Llama-3.1-8B"
MODEL_KEY = "meta-llama/Llama-3.2-3B"

mt = ModelandTokenizer(
    model_key=MODEL_KEY,
    # torch_dtype=torch.float32,
)

2024-10-22 19:35:29 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]

2024-10-22 19:35:31 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/meta-llama/Llama-3.2-3B> | size: 6127.841 MB | dtype: torch.float16 | device: cuda:0


In [21]:
from src.functional import patchscope, get_hs, logit_lens
from src.tokens import prepare_input

prompt = "The city of Paris is in the country of Italy."
inputs = prepare_input(prompt, mt)

In [22]:
mt.n_layer

28

In [23]:
h = get_hs(
    mt = mt,
    input = inputs,
    locations = [(mt.layer_name_format.format(24), inputs.input_ids.shape[1] - 1)]
)

h.shape

torch.Size([3072])

In [24]:
interested_tokens = ["True", "False", "true", "false", "yes", "no"]
mt.tokenizer(interested_tokens)

{'input_ids': [[128000, 2575], [128000, 4139], [128000, 1904], [128000, 3934], [128000, 9891], [128000, 2201]], 'attention_mask': [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]}

In [25]:
patchscope(
    mt = mt, 
    h = h,
    layer_idx = 4,
    interested_tokens = [2575, 4139],
    k = 5
)

([PredictedToken(token=' cat', prob=0.22968816757202148, logit=15.21875, token_id=8415),
  PredictedToken(token=' copy', prob=0.07456883043050766, logit=14.09375, token_id=3048),
  PredictedToken(token=' ->', prob=0.03868579864501953, logit=13.4375, token_id=1492),
  PredictedToken(token=' ', prob=0.012559440918266773, logit=12.3125, token_id=220),
  PredictedToken(token='.', prob=0.010911830700933933, logit=12.171875, token_id=662)],
 {2575: (26359,
   PredictedToken(token='True', prob=6.081589845052804e-07, logit=2.376953125, token_id=2575)),
  4139: (57773,
   PredictedToken(token='False', prob=7.946209734654985e-08, logit=0.341796875, token_id=4139))})

In [26]:
from src.functional import predict_next_token

placeholder = "cupcake"
target_prompt = f"cat -> cat; hello -> hello; Microsoft -> Microsoft; copy -> copy; Python -> Python; {placeholder} ->"

predict_next_token(
    mt = mt, 
    inputs = target_prompt
)


[[PredictedToken(token=' cup', prob=0.8846616744995117, logit=23.859375, token_id=10747),
  PredictedToken(token=' cupcakes', prob=0.041376180946826935, logit=20.796875, token_id=88993),
  PredictedToken(token=' cake', prob=0.023209931328892708, logit=20.21875, token_id=19692),
  PredictedToken(token=' Cup', prob=0.017519773915410042, logit=19.9375, token_id=11098),
  PredictedToken(token=' cat', prob=0.0012494523543864489, logit=17.296875, token_id=8415)]]